In [45]:
import spotipy
import spotipy.util as util
import os
import pandas as pd
import json

CLIENTID = os.environ.get('CLIENTID')
CLIENTSECRET = os.environ.get('CLIENTSECRET')
REDIRECT_URI = 'https://example.com/callback'

In [46]:
# Authenticate with spotipy
scope = 'user-library-read'
util.prompt_for_user_token("1235792",
                           scope,
                           client_id=CLIENTID,
                           client_secret=CLIENTSECRET,
                           redirect_uri=REDIRECT_URI)

'BQBgXuvBEOCZUawrr462hlXaeWWdOo2CS49MVDydgAkwqfeo1LuPOZW6xAlWRuvLrIwEv5_sBxqYpxNd5FUitm7yFoOJuei0iCIPdPKzBfDrtHg4C5eOJOzWBy7AfaifRPcqJtiTNxOwy-NrC4esjpox8nLJTNl58XccFdoAWDc4HSaOctAvKD8VK59dwg6b1b9IaaH-_YuRG3vEIjCoo0qZ-wWwu1UVrus'

In [47]:
scope = 'playlist-read-private'
token = util.prompt_for_user_token("1235792",
                           scope,
                           client_id=CLIENTID,
                           client_secret=CLIENTSECRET,
                           redirect_uri=REDIRECT_URI)
if token:
    sp = spotipy.Spotify(auth=token)
    results = sp.user_playlists("1235792")
    for playlist in results['items']:
        print(f"Playlist: {playlist['name']}")
        print(f"Description: {playlist['description']}")
        print(f"Number of tracks: {playlist['tracks']['total']}")
        print(f"Owner: {playlist['owner']['display_name']}")
        print(f"Link: {playlist['external_urls']['spotify']}")
        print(f"id: {playlist['id']}")
        print()
        

Playlist: Endless Summer 2025
Description: 
Number of tracks: 300
Owner: Travis Harry
Link: https://open.spotify.com/playlist/0KlZzaoXl3GtUuqA29tSgs
id: 0KlZzaoXl3GtUuqA29tSgs

Playlist: Summer 2025
Description: 
Number of tracks: 119
Owner: Travis Harry
Link: https://open.spotify.com/playlist/4XGVNo18btSbR1gG1G6CbK
id: 4XGVNo18btSbR1gG1G6CbK

Playlist: Summer Candidates - 500
Description: 
Number of tracks: 104
Owner: Travis Harry
Link: https://open.spotify.com/playlist/61UKXact4yZbhbMGIjBqIc
id: 61UKXact4yZbhbMGIjBqIc

Playlist: Summer Candidates - 300
Description: 
Number of tracks: 118
Owner: Travis Harry
Link: https://open.spotify.com/playlist/0Vs5KgGbaeZBIm8rx7VgTY
id: 0Vs5KgGbaeZBIm8rx7VgTY

Playlist: Summer Candidates - 100
Description: 
Number of tracks: 126
Owner: Travis Harry
Link: https://open.spotify.com/playlist/1pkD0BsBFqbLh9upGeCfu2
id: 1pkD0BsBFqbLh9upGeCfu2

Playlist: Summer Candidates - 10
Description: 
Number of tracks: 23
Owner: Travis Harry
Link: https://open.spot

In [48]:
scope = 'playlist-read-private'
token = util.prompt_for_user_token("1235792",
                           scope,
                           client_id=CLIENTID,
                           client_secret=CLIENTSECRET,
                           redirect_uri=REDIRECT_URI)

def get_playlist_tracks(playlist_id):
    results = sp.playlist_tracks(playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

def convert_tracks_to_df(tracks):
    df = pd.DataFrame(tracks)
    df = df[['track']]
    df['track_name'] = df['track'].apply(lambda x: x['name'])
    df['artist_name'] = df['track'].apply(lambda x: ', '.join([artist['name'] for artist in x['artists']]))
    df['track_id'] = df['track'].apply(lambda x: x['id'])
    df['release_date'] = df['track'].apply(lambda x: x['album']['release_date'])
    df['release_date'] = pd.to_datetime(df['release_date'])
    return df[['track_name', 'artist_name', 'track_id', 'release_date']].copy()


In [49]:
# Summer 2025
tracks_summer25 = get_playlist_tracks("4XGVNo18btSbR1gG1G6CbK")
summer25 = convert_tracks_to_df(tracks_summer25)
summer25.loc[summer25['release_date'] < '2022-05-01']

,track_name,artist_name,track_id,release_date
109,A Good Song Never Dies,Saint Motel,5K43AHQPTJTqlyRZ3uBGKz,2021-06-25


In [50]:
# Endless Summer 2025
tracks_endless25 = get_playlist_tracks("0KlZzaoXl3GtUuqA29tSgs")
endless25 = convert_tracks_to_df(tracks_endless25)
endless25.loc[endless25['release_date'] < '2020-05-01']

,track_name,artist_name,track_id,release_date


In [51]:
# Summers - All
tracks_allsummers = get_playlist_tracks("4M3yOOURtCF7VbPbBvHhfL")
allsummers = convert_tracks_to_df(tracks_allsummers)

In [52]:
allsummers.loc[allsummers['track_name']=="Colorado"]

,track_name,artist_name,track_id,release_date


In [53]:
# Merge allsummers with endless25 using an inner join on "track_name" and "artist_name"
merged_df = pd.merge(allsummers, endless25, on=['track_name', 'artist_name'], how='inner')
merged_df

,track_name,artist_name,track_id_x,release_date_x,track_id_y,release_date_y


In [54]:
# Group summer25 by artist_name
grouped_summer25 = summer25.groupby('artist_name').agg({
    'track_name': 'count',
    'release_date': 'max'
}).reset_index().sort_values(by='track_name', ascending=False)
grouped_summer25

,artist_name,track_name,release_date
0,2hollis,4,2025-04-04
5,Bartees Strange,3,2025-02-14
39,Lola Young,3,2024-06-21
41,Marta Del Grandi,3,2024-11-07
24,Greg Freeman,3,2022-10-15
...,...,...,...
31,"Jeshi, Fredwave, Louis Culture, J. Caesar",1,2025-01-24
30,Jax,1,2024-06-28
29,JJ,1,2025-03-06
28,"Horse Vision, Tiffi M",1,2025-03-07


In [55]:
# Group endless25 by artist_name
grouped_endless25 = endless25.groupby('artist_name').agg({
    'track_name': 'count',
    'release_date': 'max'
}).reset_index().sort_values(by='track_name', ascending=False)
grouped_endless25

,artist_name,track_name,release_date
120,Orla Gartland,9,2024-10-04
101,MJ Lenderman,6,2024-09-06
19,Bartees Strange,6,2025-02-14
142,Rosie Tucker,5,2024-03-22
60,Greg Freeman,5,2025-04-23
...,...,...,...
81,Keep Dancing Inc,1,2024-01-19
82,"Ken Yates, Kathleen Edwards",1,2022-06-03
83,"Ken Yates, Stephanie Lambring",1,2022-06-03
84,"Kendrick Lamar, Beth Gibbons",1,2022-05-13


In [56]:
# import spotipy
# from spotipy.oauth2 import SpotifyClientCredentials

# scope = 'playlist-read-private'

# auth_manager = SpotifyClientCredentials(client_id=CLIENTID, client_secret=CLIENTSECRET)
# sp = spotipy.Spotify(auth_manager=auth_manager)

# playlists = sp.user_playlists('1235792')
# while playlists:
#     for i, playlist in enumerate(playlists['items']):
#         print(f"{i + 1 + playlists['offset']:4d} {playlist['uri']} {playlist['name']}")
#     if playlists['next']:
#         playlists = sp.next(playlists)
#     else:
#         playlists = None

In [57]:
import tekore as tk

conf = (CLIENTID, CLIENTSECRET, REDIRECT_URI)
token = tk.prompt_for_user_token(*conf, scope=tk.scope.every)

Opening browser for Spotify login...


In [58]:
tek = tk.Spotify(token)
playlists = tek.playlists('1235792')
playlists

SimplePlaylistPaging with fields:
  href = 'https://api.spotify.com/v1/users/1235792/playlists?offset=0&l...'
  items = [20 x SimplePlaylist(collaborative, description, ...)]
  limit = 20
  next = 'https://api.spotify.com/v1/users/1235792/playlists?offset=20&...'
  offset = 0
  previous = None
  total = 56